In [23]:
import numpy as np
from scipy.optimize import minimize
import tkinter as tk
from tkinter import messagebox
from honeycomb_analysis import youngs_modulus_E1, youngs_modulus_E2, shear_modulus_G12, E_s

def objective(x):
    t, h, l, theta = x
    lambda_1 = 0.33
    lambda_2 = 0.33
    lambda_3 = 0.34
    E1 = youngs_modulus_E1(t, l, h)
    E2 = youngs_modulus_E2(t, l, h)
    G12 = shear_modulus_G12(t, l, h)
    return -(lambda_1 * E1 + lambda_2 * E2 + lambda_3 * G12)  # Negative for minimization


In [24]:

def constraint_alpha_lower(x):
    t, h, l, theta = x
    alpha = h / l
    return alpha - 0.5

def constraint_alpha_upper(x):
    t, h, l, theta = x
    alpha = h / l
    return 1 - alpha

def constraint_beta_lower(x):
    t, h, l, theta = x
    beta = t / l
    return beta - 0.004

def constraint_beta_upper(x):
    t, h, l, theta = x
    beta = t / l
    return 0.008 - beta

def constraint_theta_lower(x):
    t, h, l, theta = x
    return theta + np.pi / 6

def constraint_theta_upper(x):
    t, h, l, theta = x
    return np.pi / 6 - theta

def calculate_relative_density(t, h, l):
    return (2 / np.sqrt(3)) * (t / l) * (1 - (t / (2 * np.sqrt(3) * h)))

def constraint_relative_density(x):
    t, h, l, theta = x
    rho_s = calculate_relative_density(t, h, l)
    rho_ori = float(rho_ori_entry.get())
    return rho_ori - rho_s  # Ensure that rho_s <= rho_ori




In [25]:
# Combine all constraints
constraints = [
    {'type': 'ineq', 'fun': constraint_alpha_lower},
    {'type': 'ineq', 'fun': constraint_alpha_upper},
    {'type': 'ineq', 'fun': constraint_beta_lower},
    {'type': 'ineq', 'fun': constraint_beta_upper},
    {'type': 'ineq', 'fun': constraint_theta_lower},
    {'type': 'ineq', 'fun': constraint_theta_upper},
    {'type': 'ineq', 'fun': constraint_relative_density}
]

def run_optimization():
    try:
        t_lower = float(t_lower_entry.get())
        t_upper = float(t_upper_entry.get())
        h_lower = float(h_lower_entry.get())
        h_upper = float(h_upper_entry.get())
        l_lower = float(l_lower_entry.get())
        l_upper = float(l_upper_entry.get())
        theta_lower = float(theta_lower_entry.get())
        theta_upper = float(theta_upper_entry.get())

        initial_t = float(initial_t_entry.get())
        initial_h = float(initial_h_entry.get())
        initial_l = float(initial_l_entry.get())
        initial_theta = float(initial_theta_entry.get())

        bounds = [(t_lower, t_upper), (h_lower, h_upper), (l_lower, l_upper), (theta_lower, theta_upper)]
        x0 = [initial_t, initial_h, initial_l, initial_theta]

        # Validate initial guess
        initial_constraints = [
            constraint_alpha_lower(x0), constraint_alpha_upper(x0), constraint_beta_lower(x0), 
            constraint_beta_upper(x0), constraint_theta_lower(x0), constraint_theta_upper(x0), 
            constraint_relative_density(x0)
        ]
        print(f"Initial guess: {x0}")
        print(f"Initial constraints values: {initial_constraints}")

        for i, constraint_value in enumerate(initial_constraints):
            if constraint_value < 0:
                print(f"Constraint {i+1} violated: {constraint_value}")

        if any(c < 0 for c in initial_constraints):
            raise ValueError("Initial guess does not satisfy the constraints")

        solution_slsqp = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints, options={'disp': True})
        solution_cobyla = minimize(objective, x0, method='COBYLA', bounds=bounds, constraints=constraints, options={'disp': True})

        if solution_slsqp.success:
            slsqp_solution = f"SLSQP Solution: {solution_slsqp.x}\nSLSQP Objective value: {-solution_slsqp.fun}"
            print(slsqp_solution)
        else:
            slsqp_solution = f"SLSQP Optimization failed: {solution_slsqp.message}"
            print(slsqp_solution)

        if solution_cobyla.success:
            cobyla_solution = f"COBYLA Solution: {solution_cobyla.x}\nCOBYLA Objective value: {-solution_cobyla.fun}"
            print(cobyla_solution)
        else:
            cobyla_solution = f"COBYLA Optimization failed: {solution_cobyla.message}"
            print(cobyla_solution)

        result_message = f"{slsqp_solution}\n\n{cobyla_solution}"
        messagebox.showinfo("Optimization Results", result_message)

    except Exception as e:
        messagebox.showerror("Error", str(e))

# Set up the GUI
root = tk.Tk()
root.title("Optimization")

# Define and place GUI elements
tk.Label(root, text="t lower:").grid(row=0, column=0)
t_lower_entry = tk.Entry(root)
t_lower_entry.grid(row=0, column=1)
t_lower_entry.insert(0, "0.004")

tk.Label(root, text="t upper:").grid(row=1, column=0)
t_upper_entry = tk.Entry(root)
t_upper_entry.grid(row=1, column=1)
t_upper_entry.insert(0, "0.012")  # Adjusted

tk.Label(root, text="h lower:").grid(row=2, column=0)
h_lower_entry = tk.Entry(root)
h_lower_entry.grid(row=2, column=1)
h_lower_entry.insert(0, "1")

tk.Label(root, text="h upper:").grid(row=3, column=0)
h_upper_entry = tk.Entry(root)
h_upper_entry.grid(row=3, column=1)
h_upper_entry.insert(0, "50")

tk.Label(root, text="l lower:").grid(row=4, column=0)
l_lower_entry = tk.Entry(root)
l_lower_entry.grid(row=4, column=1)
l_lower_entry.insert(0, "1")

tk.Label(root, text="l upper:").grid(row=5, column=0)
l_upper_entry = tk.Entry(root)
l_upper_entry.grid(row=5, column=1)
l_upper_entry.insert(0, "10")

tk.Label(root, text="theta lower:").grid(row=6, column=0)
theta_lower_entry = tk.Entry(root)
theta_lower_entry.grid(row=6, column=1)
theta_lower_entry.insert(0, str(-np.pi/6))

tk.Label(root, text="theta upper:").grid(row=7, column=0)
theta_upper_entry = tk.Entry(root)
theta_upper_entry.grid(row=7, column=1)
theta_upper_entry.insert(0, str(np.pi/6))

tk.Label(root, text="Initial t:").grid(row=8, column=0)
initial_t_entry = tk.Entry(root)
initial_t_entry.grid(row=8, column=1)
initial_t_entry.insert(0, "0.012")  # Adjusted

tk.Label(root, text="Initial h:").grid(row=9, column=0)
initial_h_entry = tk.Entry(root)
initial_h_entry.grid(row=9, column=1)
initial_h_entry.insert(0, "3")

tk.Label(root, text="Initial l:").grid(row=10, column=0)
initial_l_entry = tk.Entry(root)
initial_l_entry.grid(row=10, column=1)
initial_l_entry.insert(0, "3")

tk.Label(root, text="Initial theta:").grid(row=11, column=0)
initial_theta_entry = tk.Entry(root)
initial_theta_entry.grid(row=11, column=1)
initial_theta_entry.insert(0, "0.2")

tk.Label(root, text="Original Relative Density (ρ_ori):").grid(row=12, column=0)
rho_ori_entry = tk.Entry(root)
rho_ori_entry.grid(row=12, column=1)
rho_ori_entry.insert(0, "0.0153")

tk.Button(root, text="Run Optimization", command=run_optimization).grid(row=13, column=0, columnspan=2)

root.mainloop()


Initial guess: [0.012, 3.0, 3.0, 0.2]
Initial constraints values: [0.5, 0.0, 0.0, 0.004, 0.7235987755982989, 0.3235987755982988, np.float64(0.010686531179816326)]
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -12.269634525983474
            Iterations: 15
            Function evaluations: 81
            Gradient evaluations: 11
SLSQP Optimization failed: Positive directional derivative for linesearch
COBYLA Solution: [0.012      1.5        1.5        0.49622881]
COBYLA Objective value: 11.90099038083806
